# Pythonプログラミング実践編
本講義では、Pythonの教科書（基礎）だけでは学べない実践で必要な考え方を学んでいきたいと思います。

主にプログラムの**汎用性**（複数のケースで利用できること）についてと、
近年注目されている生成AIについても触れてみましょう。


はじめに、本講義で使用するファイルを皆さんの環境にダウンロードするため、次のコードを実行してください。

In [ ]:
!curl -L -o meta.txt https://github.com/tendo-sms/python_beginner/raw/main/files_6/meta.txt
!curl -L -o data1.csv https://github.com/tendo-sms/python_beginner/raw/main/files_6/data1.csv
!curl -L -o data2.csv https://github.com/tendo-sms/python_beginner/raw/main/files_6/data2.csv
!curl -L -o data3.CSV https://github.com/tendo-sms/python_beginner/raw/main/files_6/data3.CSV
!curl -L -o sample.json https://github.com/tendo-sms/python_beginner/raw/main/files_6/sample.json

# ARIM事業におけるRDEの概略

<img src="https://github.com/tendo-sms/python_beginner/raw/main/files_6/figure/rde.png" width="70%">

実践の説明例として、NIMSが運用しているRDE(Research Data Express)というシステムを挙げてみます。

RDEとは、材料研究のために研究現場で日々創出される材料データを効率的に収集するために、IoT データ転送機能、データ構造化機能、データ登録機能を備えたデータ収集・管理・提供システムのことです。

本講義で注目する部分は、アップロードされたデータを処理する(=データ構造化)機能です。

構造化の処理には、この2日間で皆さんが学んできた**Python**で開発が進められています。

では、実践ではどのようなことを考える必要があるのでしょうか？

そのひとつはプログラムの**汎用性**です。

どのようなことをすればプログラムの**汎用性**の確保ができるのか紹介していきたいと思います。

実践とは言っても、今回のセミナーを通して聴講いただいた方であれば、十分に理解できる内容です。


# アップロードされたファイルの読み込み

自分のために作るプログラムは、ユーザ=自分なので知らず知らずのうちに暗黙的なルールがベースになっていることがあります。

その暗黙的なルールを取り除いていく、というのが**汎用性**を確保するコツです。

複数のユーザが利用するのであれば、汎用性の確保の重要度は増します。

まずは、ファイルを読み込んで内容を表示するだけのコードを考えてみましょう。


In [ ]:
# 読み込むファイル名を指定
input_file = "meta.txt"

# ファイルを読み込む
with open(input_file, "r") as f:
    data = f.read()

# データの表示
print(data)

無事に内容を表示することができました！

エラーは出ていないので、システム（例えばRDEなど）に配置して完成！

・・・とはならず、次の問題を考える必要がないでしょうか。

- ユーザは、meta.txtというファイル名でアップロードしてくるでしょうか？
- 処理したいファイルは.txtでしょうか？
- ユーザは、複数のファイルを同時にアップロードしてこないでしょうか？

これらを意識して、今度はcsvファイルを全て処理するように書き換えてみましょう。


In [ ]:
from pathlib import Path

# 現在のディレクトリパスを取得
cwd = Path.cwd()

for f in cwd.glob("*.csv"):
    # ファイル名を表示
    print(f"ファイル名:{f.name}")

    with open(f, "r") as f:
        print(f.read())


**glob**を使うことで.csvという拡張子のファイルを全部読み取ることができました！

ですが、人によっては以下のように表示されていないでしょうか？

```
ファイル名:data2.csv
C,D
3,4
5,6

ファイル名:data1.csv
A,B
1,2
3,4
```

もしこのように表示されていたら、**data2.csv**を先に読み込んで、後から**data1.csv**が読み込まれている状態になっています。

**glob**というのは、パターンに該当したものを見つけて列挙してくれる便利な命令ですが、実はその順番は決まっていません。

読み込む順番を気にしないのであればこのままでもよいですが、不便なこともあります。

上記の表示例のように戸惑ってしまうかもしれません。

もし最初に読み取ったデータを使って次に読み取ったデータを処理しなければならないときには問題が発生します。

```
例）
計算したい値 = 最初のファイルの1行目1列目 / 次のファイルの1行目1列目
```

では順番も意識して修正してみましょう。

In [ ]:
from pathlib import Path

# 現在のディレクトリパスを取得
cwd = Path.cwd()

# srotedを追加してソートする
for f in sorted(cwd.glob("*.csv")):
    # ファイル名を表示
    print(f.name)

    with open(f, "r") as f:
        print(f.read())


**sorted**を使うことで、読み込み順序が辞書順で保証されるようにできました!

これで複数のファイルをアップロードしたときの**汎用性**が少しアップしました。

では、念のためにディレクトリにどのようなファイルがあるか確認してみましょう。

In [ ]:
# ディレクトリの一覧を確認
!ls

おっと、先ほど表示されていない**data3.CSV**というファイルがありますね…。

同じCSVファイルのはずなのに、なぜ読み込まれなかったのでしょうか。

気が付かれた方もいるかもしれませんが、ファイルの拡張子は小文字（.csv）と大文字（.CSV）の違いがあったようです。

プログラムでは、大文字・小文字は正確に区別されます。

globの中の*.csvでは小文字しか対象にできていなかったということです。

では大文字・小文字を区別せずに処理するために、次のように修正してみましょう。

In [ ]:
from pathlib import Path

# 現在のディレクトリパスを取得
cwd = Path.cwd()

# .csvでも.CSVでも.Csvでも・・・処理対象にする
for f in sorted(cwd.glob("*.[Cc][Ss][Vv]")):
    # ファイル名を表示
    print(f.name)

    with open(f, "r") as f:
        print(f.read())


3つのCSVファイルを無事に読み込むことに成功しました！

また少し汎用性を向上することができました。

globの引数はパターンを記述しますが、正規表現ではなく**標準Unixパス拡張ルールに準拠**というのが採用されています。

興味のある方は下記の公式ページを確認してみてください。

https://docs.python.org/ja/3.13/library/glob.html

では、新たなCSVファイル（data4.csv）も問題がないか試してみましょう。

data4.csvファイルをダウンロードして、同じように実行してみます。


In [ ]:
!curl -L -o data4.csv https://github.com/tendo-sms/python_beginner/raw/main/files_6/data4.csv

In [ ]:
# さきほどと全く同じコードを流用
from pathlib import Path

# 現在のディレクトリパスを取得
cwd = Path.cwd()

# .csvでも.CSVでも.Csvでも・・・処理対象にする
for f in sorted(cwd.glob("*.[Cc][Ss][Vv]")):
    # ファイル名を表示
    print(f.name)

    with open(f, "r") as f:
        print(f.read())


エラーが出てしまいました…。

このエラーはファイルエンコーディングのエラーです。

「パッケージとモジュール・ファイル操作」の講義で出てきましたが、覚えていますでしょうか？

ファイルをオープンするとき、encodingオプションを指定しない場合、その動作環境のデフォルト設定を使用します。
（この環境では、UTF-8が使われています。）

アップロードするエンコーディングをユーザに守ってもらうことで防ぐことはできます。

ここでは、便利なライブラリとしてcharset-normalizerを使って自動判定させてみましょう。

charset-normalizerは標準パッケージではないので、pipでインストールしてください。

In [ ]:
!pip install charset-normalizer

In [ ]:
from pathlib import Path
from charset_normalizer import from_bytes

# 現在のディレクトリを取得
cwd = Path.cwd()

# .csvでも.CSVでも.Csvでも・・・処理対象にする
for f in sorted(cwd.glob("*.[Cc][Ss][Vv]")):
    # エンコーディングを取得
    with open(f, "rb") as ff:
        enc = from_bytes(ff.read()).best().encoding

    # ファイル名とエンコーディングを表示
    print(f"{f.name}のファイルのエンコーディング: {enc}")

    with open(f, "r", encoding=enc) as ff:
        print(ff.read())

今度はエラーなく表示されたようです。

エンコーディングによる問題は、日本語が含まれる場合によく発生します。

WindowsユーザがExcelで作成したcsvファイルは通常、cp932（「パッケージとモジュール・ファイル操作」で紹介）と呼ばれる形式になります。

**!!! 重要 !!!**

ファイルの文字エンコーディングを100%確実に判定する方法は存在しません。

charset-normalizer のようなライブラリは、ファイルの中身（バイト列）を分析して、もっとも可能性の高いエンコーディングを「推測」しているだけです。

ファイル内の文字数が少なかったり、使われている文字が限られていたりすると、推測に必要な手がかりが不足して、たとえエラーが出なくても誤ったエンコーディングが選ばれる可能性があります。

そのため、エンコーディングの自動判定は「正しそうに見えても100%正しいとは限らない」ことを前提にご使用ください。


以上で最初と比較してファイル読込みの汎用性は高くなったのではないでしょうか。


# 生成AI(ChatGPT)について

2022年11月にChatGPTが公開され、世界に大きな衝撃を与えました。

インターネットやスマートフォンに続く技術革命とも称される生成AIは、公開から2年以上が経過した現在、さまざまな分野に活用されるようになり、特にプログラミング分野で高い適性を示しています。

ChatGPTが登場する以前は、「プログラミングの知識がなくてもビジュアル操作だけでアプリケーションが作れる」というノーコード／ローコードのアプローチが注目されていました。

しかし現在では、AIによるコーディング支援の普及により、多くのユーザーがAIと連携してプログラミングに取り組むスタイルにシフトしているように見受けられます。

本講義では、ChatGPTを利用してどのようにコードを作成していくかを体験してみましょう。

# ChatGPTによるアップロードされたファイルの読み込み（一から作ってみよう）

さて、皆さんならどのようにChatGPTに聞いてみますか？

実際にChatGPTを使ってみましょう！（3分程度）

https://chatgpt.com/


例）

`複数の人が利用するシステムで、アップロードされたファイルをPythonを使って読み込んで表示させたいです。`




色々なライブラリ（Djangoやflask？）を紹介されたり、関数なども盛り込まれていませんでしたか？

何度か会話すれば近づくことはできるかもしれませんが、講義の例を一発で教えてもらうのは困難かもしれません。

少し工夫してみて試してみます。

```
以下があなたの属性です。
# Pythonのプロです。
# 初心者プログラマーの教育経験が豊富です。

以下が課題です。
Pythonでローカルにあるcsvファイルを読み込もうとしています。
汎用性を高いコードを作ってください。
自作関数は使わずに、グローバル領域で書いてください。
できるだけ短く余計なライブラリは使わないでください。
```



# ChatGPTによるアップロードされたファイルの読み込み（ブラッシュアップ）

講義で示したコードをChatGPTをブラッシュアップしてもらいましょう！（2分程度）

In [ ]:
from pathlib import Path
from charset_normalizer import from_bytes

# 現在のディレクトリを取得
cwd = Path.cwd()

# .csv / .CSV / .Csv などをすべて対象にする
for csv_file in sorted(cwd.glob("*.[Cc][Ss][Vv]")):
    try:
        # バイナリ読み込みでエンコーディング判定
        with open(csv_file, "rb") as bin_f:
            result = from_bytes(bin_f.read())
            if not result:
                print(f"{csv_file.name} のエンコーディングを判別できません。")
                continue
            enc = result.best().encoding

        print(f"{csv_file.name} のファイルのエンコーディング: {enc}")

        # 推定エンコーディングで読み込み
        with open(csv_file, "r", encoding=enc, errors="replace") as text_f:
            print(text_f.read())

    except Exception as e:
        print(f"{csv_file.name} の読み込み中にエラーが発生しました: {e}")


以上のように、一からChatGPTに質問すると色んなライブラリを使って中級者レベルのコードを示してもらえることが多いです。

一方で自分が書きたいコードの方向性やレベルを維持したいのなら、ベースになるコードを示した上で進めるとよいかもしれません。

以上のように生成AIはプログラミングをする上で強力なアシスタントツールです。

ぜひ活用してみましょう。

ただし、生成AIサービスは情報漏洩などのリスクがありますので、業務や研究で使用する場合は所属機関のルールに従うようにしましょう。

また、AIの提案がいつも完璧とは限りません。より良い活用のためにも、自分自身でも知識を深めていくことが大切です。

# 日付・日時のパース（構文解析）

ご存じの通り、世界には日付の表現方法は様々なものがあります。

もし表示形式がわかっていれば、**datetime**モジュールを使用することで文字列から日付のオブジェクトに変換することが簡単にできます。

In [ ]:
from datetime import datetime

# 年.月.日の場合
date1 = "2021.05.29"
print(f"{date1} -> {datetime.strptime(date1, '%Y.%m.%d')}")

# 年/月/日の場合
date2 = "2021/05/29"
print(f"{date2} -> {datetime.strptime(date2, '%Y/%m/%d')}")


文字列から日付のオブジェクトに変換することができました。

ここで、%Yや%mというのはディレクティブと呼ばれ、それぞれに応じた意味があります。

代表的な例をいくつかご紹介します。

|ディレクティブ|意味|
|----|----|
|%Y|西暦 ( 4桁) の 10 進表記を表します。|
|%m|0埋めした10進数で表記した月。|
|%d|0埋めした10進数で表記した月中の日にち。|
|%H|0埋めした10進数で表記した時 (24時間表記)。|
|%M|0埋めした10進数で表記した分。|
|%S|0埋めした10進数で表記した秒。|

さて、**datetime.strptime**で指定する場合は、その文字列の形式がすでにわかっている場合にのみ有効な方法です。

ではもし、形式がわからない場合はどうすればよいでしょうか。

ここで活躍するのが、**dateutil**パッケージです。

実際に上記の例で使ってみましょう。



In [ ]:
from dateutil import parser

# 年.月.日の場合
date1 = "2021.05.29"
print(f"{date1} -> {parser.parse(date1)}")

# 年/月/日の場合
date2 = "2021/05/29"
print(f"{date2} -> {parser.parse(date2)}")

うまく変換できましたね！

この方法がベストなのか、生成AIを使って質問してみましょう。（1分程度）

それでは、どのような文字列でも変換可能なのでしょうか。

次のように色んなパターンを確かめてみましょう。


In [ ]:
from dateutil import parser

date_lst = [
    "2021.05.29",
    "2021-05-29T14:17:27Z",
    "2021-05-29 15:17:27.133860",
    "2021-05-29 16:17:27.133860+00:00",
    "2021-05-29 17:17:27.133860+05:00",
    "2021/05/29",
    "2021/05/29T14:17:27Z",
    "2021/05/29 15:17:27.133860",
    "2021/05/29 16:17:27.133860+00:00",
    "2021/05/29 17:17:27.133860+05:00",
    "May 29 2022 12:17PM",
    "May 29 2022 at 9:17PM",
    "May 29, 2022, 19:17:27",
    "Tue, 05-29-2022, 1:17AM",
    "Tue, 29 May, 2022",
    "Tuesday, 29th May, 2022 at 12:17p",
    "2022/02/ 7 14:04:45"
]

for d in date_lst:
    print(f"{d} -> {parser.parse(d)}")

ほとんどの場合、うまくいったようですが「"2022/02/ 7 14:04:45"」はエラーが出てしまったようです。

ではエラーメッセージをChatGPTに貼り付けてみましょう。

何も問題なく読み取れる気がしますが、実はこの文字列は日の前に空白スペースがあります。

それが原因で日付として認識できなかったようです。

このような場合は次のように細工してから実行してみましょう。




In [ ]:
from dateutil import parser

date = "2022/02/ 7 14:04:45"

# 文字列を修正　2022/02/ 7 14:04:45　-> 2022/02/07 14:04:45
new_date = date.replace("/ ", "/0")
print(f"{date} -> {new_date} -> {parser.parse(new_date)}")


今度はうまく変換できたようです。

このように、基本的には**dateutil**に頼っていますがイレギュラーなものだけは別途加工してから利用することもあります。

日付を認識するために、色んな形式をサポートできるといいですね。

これも**汎用性**の確保において重要です。


# JSONファイルの入出力

辞書の取り扱いの中で、JSONファイルについて少し触れていたかと思います。

JSONはその扱いやすさから設定ファイルやWeb上でのデータのやり取りなどに利用されています。

RDEでも、JSONは利用されていてその構文は「キー:値」というPythonの辞書のような形式です。

サンプルのファイル「sample.json」の中身は以下のように記述されています。

```
{
  "A":1,
  "B":2
}
```

これらのJSONファイルを読み込むには、カッコ{}を自分で解釈しながら変数に入れる必要はありません。

Pythonが標準で提供している**json**モジュールを使うと簡単に読むことができます。

In [ ]:
import json

# 読み込むファイル名を指定
input_file = "sample.json"

# ファイルを読み込む
with open(input_file, "r") as f:
    data = json.load(f)

# データの表示
print(data)
print(type(data))

**json.load**という関数を呼び出すだけで、Pythonの辞書型として解釈されて読み込むことができました。

では、この**data**に何か追加して保存してみましょう。


In [ ]:
# 辞書にデータを追加
data["あいうえお"] = "かきくけこ"

# JSONで出力
with open("new_sample.json", "w") as f:
    json.dump(data, f)

ではどのような出力データになったのか、左のメニューからnew_sample.jsonを確認してみましょう。


`{"A": 1, "B": 2, "\u3042\u3044\u3046\u3048\u304a": "\u304b\u304d\u304f\u3051\u3053"}`


JSONではありますが、何かおかしな点があると思います。

横長であり、日本語がよくわからない形式になっています。

横長は見づらいだけで、JSONとしては間違ってはいません。

日本語は\u〇〇〇というように変換されてしまっています。

**json**モジュールでは、不十分なのでしょうか。

いえ、これらを解消するためには2つオプションを追加すれば大丈夫です。


In [ ]:
# 辞書にデータを追加
data["あいうえお"] = "かきくけこ"

# JSONで出力 オプションindentとensure_asciiを追加
with open("new_sample.json", "w") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

今度は、いかがでしょうか？

次のように書き込まれていることが確認できたと思います。

```
{
    "A": 1,
    "B": 2,
    "あいうえお": "かきくけこ"
}
```

このように、各種ライブラリには様々なオプションがあり、何も指定しない場合は予期せぬ動作になったりします。

ですが、ライブラリが提供しているオプションによって解決する場合が多いです。

まずは、ライブラリの使い方を調べることをお勧めいたします。


# 特定行の見つけ方（inとstartswith）

テキストデータから、特定の行を見つけて処理をしたいことがあります。

ここでは、どうやって特定の行を探してきて処理するのか確かめてみましょう。

まずは、先ほども使用した**meta.txt**を表示させてみます。

In [ ]:
# シェルコマンドでテキストの中身を表示
!cat meta.txt

このファイルには、「項目名:値」という形式でデータが保存されています。

では、**"説明"**という項目を抽出してみたいと思います。

まずは、その文字列が含まれているかどうかを**in**を使って判定してみようと思います。

In [ ]:
# 読み込むファイル名を指定  変更したい場合は書き直す
input_file = "meta.txt"

# ファイルを読み込む
with open(input_file, "r") as f:
    data = f.readlines()

# ループで行ごとに調査する
for d in data:
    # もし説明という文字が入っていたら表示する
    if "説明" in d:
        print(d.rstrip())

メモの項目も抽出されてしまったようです。

原因は、値の方に**説明**という文字が入っているからです。

「ガイダンスで**説明**するもの」

**in**は、その文字列（この場合は行全体）で含まれているかをチェックしています。

当然、値の方であっても該当文字列があればヒットしてしまいます。

では、項目はかならず左側にあると考えることができそうなので、**startswith**を使ってみます。


In [ ]:
# 読み込むファイル名を指定  変更したい場合は書き直す
input_file = "meta.txt"

# ファイルを読み込む
with open(input_file, "r") as f:
    data = f.readlines()

# ループで行ごとに調査する
for d in data:
    # もし説明という文字から開始していたら表示する
    if d.startswith("説明"):
        print(d.rstrip())

今度はうまく抽出できたようです！

このように、ある程度そのデータの特徴をつかんでから適切な処理を考えないとうまくいかないケースが出てきます。

**in**を使えば文字列を抽出できると安易に考えがちですが、十分でないことも多いです。

どんな内容のデータが保存されているのかわからない、ということを念頭に置いて開発することが汎用性を高めます。

とはいえ、実際に遭遇してからでないと気が付かないことも多いです。


# おわりに

いかがでしたでしょうか？

実践に必要なコツを少しばかりご体験していただけたでしょうか。

複数のユーザが利用するプログラムでは、様々なケースに対応できる**汎用性**のあるプログラムが必要になってきます。

「ユーザ=自分」と「ユーザ≠自分」では、考慮すべきポイントが多かったと感じたのではないでしょうか。

実践編は以上となります。